---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [4]:
import pandas as pd
import numpy as np
from itertools import groupby
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [28]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ],
    columns=["State", "RegionName"]  )
    The following cleaning needs to be done:
    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    # create an empty list
    university_towns_list = []
    
    # open and store university_towns.txt
    university_towns_txt = open('university_towns.txt')
    
    # apply loop for filtering
    for line in university_towns_txt:
        if 'edit' in line:
            current_state = line.split('[')[0].strip()
        else:
            university_towns_list.append((current_state, line.split('(')[0].strip()))
    
    # set and store as a dataframe
    university_towns_df = pd.DataFrame(university_towns_list)
    
    # set column names to respective names
    university_towns_df.columns = ['State','RegionName']
    
    return university_towns_df
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
...,...,...
512,Wisconsin,River Falls
513,Wisconsin,Stevens Point
514,Wisconsin,Waukesha
515,Wisconsin,Whitewater


In [8]:
# Global

# read GDP Levels excel file
GDP = pd.read_excel('gdplev.xls', skiprows=7)

# rename column names
GDP = GDP[['Unnamed: 4', 'Unnamed: 6']].rename(columns={'Unnamed: 4': 'Quarter', 'Unnamed: 6':'GDP'})

# create Year column
GDP['Year'] = GDP['Quarter'].str[:4].astype(int)

# create a column per yearly difference
GDP['Diff'] = GDP['GDP'].diff()

# filter only from and beyond the year 2000
GDP = GDP[GDP['Year']>=2000]

# reset index
GDP = GDP.reset_index()

# stores negatives/drop in GDP per year
GDP_dec = GDP[GDP['Diff'] < 0]

# stores positives/increase in GDP per year
GDP_inc = GDP[GDP['Diff'] > 0]


In [18]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3''' 
    
    # store difference of index column between GDP Decrease
    GDP_dec['Index diff'] = GDP_dec['index'].diff()
    
    # selects index with the lowest values
    aloc = GDP_dec['Index diff'].idxmin()
    
    # returns the index with the lowest value and returns the index before it
    ans = GDP['Quarter'].iloc[aloc-1]
    
    return ans

get_recession_start()

C:\Users\Gilgamesh\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


'2008q3'

In [24]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    # store difference of index column between GDP Increase
    GDP_inc['Index diff'] = GDP_inc['index'].diff()
    
    # selects index with the highest values
    aloc = GDP_inc['Index diff'].idxmax()
    
    # returns the index with the highest value and returns the index after it
    ans = GDP['Quarter'].iloc[aloc+1]
    
    return ans

get_recession_end()

C:\Users\Gilgamesh\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


'2009q4'

In [27]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    # locate start recession
    GDP_dec_start = get_recession_start()
    
    # locate end recession
    GDP_inc_start = get_recession_end()
    
    # from start and end, set quarter as index
    GDP_index = GDP.set_index(['Quarter']).loc[GDP_dec_start:GDP_inc_start]
    
    # returns the lowest value index
    ans = GDP_index['GDP'].idxmin()
    
    return GDP_index

get_recession_bottom()

C:\Users\Gilgamesh\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,index,GDP,Year,Diff
Quarter,,,,
2008q3,246,14891.6,2008,-71.8
2008q4,247,14577.0,2008,-314.6
2009q1,248,14375.0,2009,-202.0
2009q2,249,14355.6,2009,-19.4
2009q3,250,14402.5,2009,46.9
2009q4,251,14541.9,2009,139.4


In [55]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    # read and store City Zhvi Homes csv
    housing_df = pd.read_csv('City_Zhvi_AllHomes.csv')
    
    # drop unnecessary columns from and before 2000
    housing_df = housing_df.drop(housing_df.loc[:,'1996-04':'1999-12'], axis =1)
    
    # drop other unnecessary columns
    housing_df = housing_df.drop(housing_df.columns[[0,3,4,5]], axis=1)
    
    # map housing state from states series
    housing_df['State'] = housing_df['State'].map(states)
    
    # set multi indices and sort 
    housing_df = housing_df.set_index(['State', 'RegionName']).sort_index()
    
    # change date column names into date and time
    housing_df.columns = pd.to_datetime(housing_df.columns)
    
    # resample by quarter and its mean
    housing_df = housing_df.resample('Q',axis=1).mean()
    
    # rename dates into quarterly periods
    housing_df.columns = housing_df.columns.to_period('Q').strftime('%Yq%q')
    
    return housing_df

convert_housing_data_to_quarters()

2000q1         2000q2         2000q3  \
State   RegionName                                                 
Alabama Adamsville    69033.333333   69166.666667   69800.000000   
        Alabaster    122133.333333  123066.666667  123166.666667   
        Albertville   73966.666667   72600.000000   72833.333333   
        Arab          83766.666667   81566.666667   81333.333333   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        101533.333333  104566.666667  108366.666667   
        Casper        89233.333333   89600.000000   89733.333333   
        Cheyenne     116866.666667  120033.333333  121533.333333   
        Evansville   128033.333333  128766.666667  130833.333333   
        Pine Bluffs   93733.333333   95066.666667   94633.333333   

                            2000q4         2001q1         2001q2  \
State   RegionName                                                 
Alabama Adamsville    71966.666667   73466.666667   74000.000000   
        Alabaster    123700.000000  123233.333333  125133.333333   
        Albertville   74200.000000   75900.000000   76000.000000   
        Arab          82966.666667   84200.000000   84533.333333   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        113000.000000  115833.333333  117200.000000   
        Casper        93166.666667   95500.000000   97633.333333   
        Cheyenne     123633.333333  125533.333333  126300.000000   
        Evansville   132066.666667  130566.666667  131433.333333   
        Pine Bluffs   98066.666667  103233.333333  104600.000000   

                            2001q3         2001q4         2002q1  \
State   RegionName                                                 
Alabama Adamsville    73333.333333   73100.000000   73333.333333   
        Alabaster    127766.666667  127200.000000  127300.000000   
        Albertville   72066.666667   73566.666667   76533.333333   
        Arab          81666.666667   83900.000000   87266.666667   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        117800.000000  117633.333333  117333.333333   
        Casper        99433.333333  100633.333333  101733.333333   
        Cheyenne     126466.666667  128133.333333  128466.666667   
        Evansville   132400.000000  133466.666667  133300.000000   
        Pine Bluffs  106500.000000  104066.666667  102233.333333   

                            2002q2  ...         2014q2         2014q3  \
State   RegionName                  ...                                 
Alabama Adamsville    73133.333333  ...   77066.666667   75966.666667   
        Alabaster    128000.000000  ...  147133.333333  147633.333333   
        Albertville   76366.666667  ...   84033.333333   84766.666667   
        Arab          87700.000000  ...  113366.666667  111700.000000   
        Ardmore                NaN  ...  140533.333333  139566.666667   
...                            ...  ...            ...            ...   
Wyoming Burns        117233.333333  ...  168866.666667  161933.333333   
        Casper       101533.333333  ...  175766.666667  177300.000000   
        Cheyenne     129633.333333  ...  177466.666667  176733.333333   
        Evansville   131066.666667  ...  296733.333333  305666.666667   
        Pine Bluffs  103566.666667  ...  148666.666667  154366.666667   

                            2014q4         2015q1         2015q2  \
State   RegionName                                                 
Alabama Adamsville    71900.000000   71666.666667   73033.333333   
        Alabaster    148700.000000  148900.000000  149566.666667   
        Albertville   86800.000000   88466.666667   89500.000000   
        Arab         111600.000000  110166.666667  109433.333333   
        Ardmore

In [65]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    # university towns
    uni_towns = get_list_of_university_towns()
    
    # housing data csv
    housing = convert_housing_data_to_quarters()
    
    # recession start
    recession_start = get_recession_start()
    
    # recession bottom
    recession_bottom = get_recession_bottom()
    
    # recession end
    recession_end = get_recession_end()
    
    # university towns
    uni_index = get_list_of_university_towns().set_index(['State', 'RegionName'])
    
    # get mean of between the dates of recession and store their mean
    housing = (housing.loc[:,recession_start] / housing.loc[:,recession_end]).to_frame('Mean').dropna()
    
    # inner merge data frames between and with housing df and university town index
    unihousing_df = uni_index.merge(housing, how='inner', left_index=True, right_index=True)
    
    # right merge data frames between house not within the university 
    nonunihousing_df = uni_index.merge(housing, how='right', left_index=True, right_index=True)
    p_value = ttest_ind(unihousing_df.values, nonunihousing_df.values).pvalue
    if p_value < 0.01:
        different = True
    else:
        different = False
    
    if unihousing_df.mean().values < nonunihousing_df.mean().values:
        better = 'university town'
    else:
        better = 'non-university town'
    ans = (different, p_value[0], better)
    return ans    
     
run_ttest()

C:\Users\Gilgamesh\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


(True, 0.0027567593349356262, 'university town')